In [10]:
import numpy as np
import torch
import tensorflow as tf
import nd2
from dask import delayed

In [ ]:
print('PyTorch: {}'.format(torch.__version__))
print('Numpy: {}'.format(np.__version__))
print('TensorFlow: {}'.format(tf.__version__))

In [ ]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']
get_available_gpus()

In [ ]:
torch.cuda.is_available()

In [2]:
def torch_fft2(data: np.ndarray):
    data2 = torch.fft.fft2(data)    
    return torch.fft.ifft2(data2)

def numpy_fft2(data: np.ndarray):
    data2 = np.fft.fft2(data)
    return np.fft.ifft2(data2)

def tf_fft2(data: np.ndarray):
    data2 = tf.signal.fft2d(data)
    return tf.signal.ifft2d(data2)

In [7]:
# Load dataset
def load_data_xarr(file:str = "../data/A1_s3001.nd2"):
    data = nd2.imread(file, xarray=True, dask=True)
    return data

def load_data(file:str = "../data/A1_s3001.nd2"):
    data = nd2.imread(file, xarray=False, dask=True)
    return data


In [8]:
data = load_data()
data

dask.array<_dask_block, shape=(5000, 512, 512), dtype=uint16, chunksize=(1, 512, 512), chunktype=numpy.ndarray>

In [13]:
%%time
for image in data:
    np_out = delayed(numpy_fft2)(image)
    result.append(np_out)


CPU times: total: 2.05 s
Wall time: 2.05 s


In [15]:
result.compute()

AttributeError: 'list' object has no attribute 'compute'

In [ ]:
data[0]

In [ ]:
%%time
# tensor = torch.from_numpy(data)
for image in data:
    print(type(image))
    # torch_out = torch_fft2(image)

In [ ]:
%%time
# tensor = tf.convert_to_tensor(data)
for image in data:
    tensor = tf.convert_to_tensor(image)
    tf_out = tf_fft2(tensor)

In [ ]:
np.testing.assert_allclose(np_out, torch_out.numpy(), rtol=1e-5)
np.testing.assert_allclose(np_out, tf_out.numpy(), rtol=1e-5)